### Thinking1
常用的文本分类方法都有哪些

答：有传统的机器学习方法，如决策树、SVM、KNN，也有深度学习方法，如TextCNN（将Text的词向量拼接成形如一个channel的图，然后使用CNN）、Bi-LSTM（将Text按正序和反序输入的结果进行拼接，相较单向的LSTM能更好地获取上下文信息）、基于双向Transformer构建的BERT。

### Thinking2
RNN为什么会出现梯度消失

答：由于RNN损失函数L对参数W求导的结果为：$$ \frac{\partial L}{\partial w} = \sum_{t=1}^T\frac{\partial L}{\partial y_T}
\frac{\partial y_T}{\partial o_T}\frac{\partial o_T}{\partial S_T}\frac{\partial S_T}{\partial S_t}\frac{\partial S_t}{\partial w} $$
其中$$ \frac{\partial S_T}{\partial S_t} = \prod_{k=t+1}^T\frac{\partial S_k}{\partial S_{k-1}} $$
而由于$$ S_k = tanh(W_xX_k+W_sS_{k-1}+b) $$
$$ tanh'\in (0,1] $$
所以$$ \frac{\partial S_T}{\partial S_t} = \prod_{k=t+1}^Ttanh'W_s $$ 在$W_s$有界且T很大时，很容易就非常接近于0，从而导致了梯度消失。

### Action1
cnews 中文文本分类：
由清华大学根据新浪新闻RSS订阅频道2005-2011年间的历史数据筛选过滤生成
训练集 50000
验证集 5000
测试集 10000
词汇（字） 5000
10个分类，包括：'体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐'

In [ ]:
# 数据探索
filename = 'cnews.train.txt'
with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()
lines[:2]

In [ ]:
# 将数据集变成小样本
num_cat = {}
num_max = 100 # 每一类最多采100个

contents, labels = [], []
with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        label, content = line.strip().split('\t')
        #print(label)
        if content:
            if label not in num_cat:
                num_cat[label] = 1
                contents.append(content)
                labels.append(label)
            else:
                if num_cat[label] < num_max:
                    num_cat[label] = num_cat[label] + 1
                    contents.append(content)
                    labels.append(label)

# 写文件
with open('cnews.train.small.txt', 'w', encoding='utf-8', errors='ignore') as f:
    for content, label in zip(contents, labels):
        f.write(label + '\t' + content+'\n')
    f.close()
print(len(contents))
print(contents[0])
print(labels[0])
print(num_cat)

#### 模型定义

In [3]:
# 定义模型
import torch
from torch import nn

class TextRNN(nn.Module):   
    def __init__(self):
        super(TextRNN, self).__init__()
        # 三个待输入的数据
        self.embedding = nn.Embedding(5000, 64)  # 进行词嵌入，5000个词，每个64维
#         self.rnn = nn.LSTM(input_size=64, hidden_size=128, num_layers=2, bidirectional=True) # bidirectional指双向的LSTM,输出为128*2维
        self.rnn = nn.GRU(input_size=64, hidden_size=128, num_layers=2, bidirectional=True)
        self.f1 = nn.Sequential(nn.Linear(256,128),
                                nn.Dropout(0.8),
                                nn.ReLU())
        self.f2 = nn.Sequential(nn.Linear(128,10),
                                nn.Softmax(dim=1))
        self.dropout = nn.Dropout(p=0.8)
 
    def forward(self, x):
        out = self.embedding(x)
        out,_ = self.rnn(out)
        out = self.dropout(out)
        out = self.f1(out[:,-1,:]) #相当于return_sequences=False
        out = self.f2(out)
        return out

In [11]:
from torch import optim
import time

def train(epochs=100):
    model = TextRNN().cuda()
    # 定义损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # 定义优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    best_val_acc = 0
    for epoch in range(epochs):
        start = time.time()
        sum_loss = 0
        # 分批训练
        for step, (x_batch, y_batch) in enumerate(train_loader):
            x = x_batch.cuda()
            y = y_batch.cuda()
            # 前向传播
            out = model(x)
            loss = Loss(out, y)
            sum_loss += loss.item()
#             print('loss=', loss)
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
#             print('accuracy:', accuracy)
            
        # 对模型进行验证
        if epoch % 5 == 0:
            sum_acc = 0
            for step, (x_batch, y_batch) in enumerate(val_loader):
                x = x_batch.cuda()
                y = y_batch.cuda()
                # 前向传播
                out = model(x)
                sum_acc += np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
            if sum_acc > best_val_acc:
                torch.save(model, 'model.pkl')
                best_val_acc = sum_acc
                print('model.pkl saved')
        
        print('epoch{} sum of loss{: .4f} time{: .4f}'.format(epoch, sum_loss, time.time()-start))

In [4]:
def test(model):
    model.eval() # 固定参数
    correct, total = 0, 0
    for x_batch, y_batch in test_loader:
        x = x_batch.cuda()
        y = y_batch.cuda()
        # 前向传播
        out = model(x)
        # 预测结果
        _, pred = torch.max(out.data, dim=1)# 取评分最高的类(dim=0是每列的最大值，dim=1是每行的最大值，返回两个值，一个是最大值tensor组一个是最大值所在的位置)
        _, label = torch.max(y.data, dim=1)
        # 对比结果与真实值
        total += y.size()[0]
        correct += (pred==label).sum().item()
    print('测试准确率为：{: .4f}%'.format(100*correct/total))          

#### 载入数据

In [5]:
# 设置数据目标文件名
train_file = 'cnews.train.txt'
test_file = 'cnews.test.txt'
val_file = 'cnews.val.txt'
vocab_file = 'cnews.vocab.txt'

In [6]:
from cnews_loader import read_vocab, read_category, process_file
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
print(categories)
# 获取训练文本中所出现过的字及其对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
# print(words)
# 获取训练书记每一个字的id和对应标签的one_hot编码
x_train, y_train = process_file(train_file, word_to_id, cat_to_id, 600)
print(x_train.shape)
print('x_train:', x_train)
x_val, y_val = process_file(val_file, word_to_id, cat_to_id, 600)
x_test, y_test = process_file(test_file, word_to_id, cat_to_id, 600)

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]
['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
(50000, 600)
x_train: [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [12]:
import numpy as np
import torch.utils.data as Data
# 设置GPU
cuda = torch.device('cuda')
x_train, y_train = torch.LongTensor(x_train), torch.Tensor(y_train)
x_val, y_val = torch.LongTensor(x_val), torch.Tensor(y_val)
x_test, y_test = torch.LongTensor(x_test), torch.Tensor(y_test)

train_dataset = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_dataset = Data.TensorDataset(x_val, y_val)
val_loader = Data.DataLoader(dataset=val_dataset, batch_size=64)
test_dataset = Data.TensorDataset(x_test, y_test)
test_loader = Data.DataLoader(dataset=test_dataset, batch_size=64)
train()

model.pkl saved
epoch0 sum of loss 562.8416 time 180.9854
epoch1 sum of loss 559.7274 time 175.8596
epoch2 sum of loss 559.2398 time 175.7991
epoch3 sum of loss 558.9432 time 175.6949
epoch4 sum of loss 558.6346 time 175.8098
model.pkl saved
epoch5 sum of loss 558.6075 time 181.1315
epoch6 sum of loss 558.4473 time 174.9646
epoch7 sum of loss 558.4816 time 174.9964
epoch8 sum of loss 558.3392 time 175.0742
epoch9 sum of loss 558.2554 time 175.0573
model.pkl saved
epoch10 sum of loss 558.2256 time 180.4739
epoch11 sum of loss 558.1895 time 130.4348
epoch12 sum of loss 558.0478 time 36.5736
epoch13 sum of loss 557.9568 time 36.5107
epoch14 sum of loss 557.8954 time 36.4607
model.pkl saved
epoch15 sum of loss 557.8702 time 37.5322
epoch16 sum of loss 557.8825 time 36.4056
epoch17 sum of loss 557.6916 time 36.4302
epoch18 sum of loss 557.7475 time 36.4511
epoch19 sum of loss 557.6788 time 36.3253
model.pkl saved
epoch20 sum of loss 557.6378 time 37.4997
epoch21 sum of loss 557.5925 time 36

In [14]:
best_model = torch.load('model.pkl')
test(best_model)

测试准确率为： 40.6500%
